In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Lasso
from sklearn.preprocessing import MinMaxScaler; scaler = MinMaxScaler()
from sklearn.metrics import mean_squared_error
from math import sqrt

Notre but est de voir si, avec la base de données dont on dispose, il existe un <b>type de cancer</b> en particulier qui serait mieux expliqué par les indicateurs socio-économiques. On s'attend à ce que certains types de cancers soient héréditaires (ex: cancer des seins), alors que d'autres pourraient etre impactés plus par des facteurs liées à l'environnment ou au style de vie (ex: cancer des poumons, mélanome). 

Par la suite, on pourrait croire que les facteurs d'environnement et de style de vie auraient un <b>impact décalé dans le temps</b> sur la mortalité, donc on essayera d'expliquer le taux par les facteurs du passé.  

#### Choix de type de cancer
On part d'un dataset contenant le taux de mortalité calculé par tranche d'âge et par sexe, pour chaque pays et pour chaque année. 
Les valeurs manquantes des indicateurs ont été remplacées par la dernière année disponible par pays. Le fichier <i>
df_sub_choix.csv</i> est issu du programme "Traitement base de données" et il comprend :
    - une séléction d'années avec un trend stable : 16 années de 2000 à 2015
    - tous les pays en commun entre Mortality et WorldBank, pour les années prises en compte : 101 pays
    - tous les types de cancer : 89 types
    - une séléction d'indicateurs : 44 choisis parmis ceux avec moins de 25% de valeurs manquantes

<table><tr><th>Category</th><th>Indicator code</th><th>Indicator description</th></tr><tr><td>Education</td><td>SE.PRM.DURS</td><td>Primary education, duration (years)</td></tr><tr><td>Education</td><td>SE.PRE.DURS</td><td>Preprimary education, duration (years)</td></tr><tr><td>Education</td><td>SE.SEC.DURS</td><td>Secondary education, duration (years)</td></tr><tr><td>Environnement</td><td>SP.URB.TOTL.IN.ZS</td><td>Urban population (% of total)</td></tr><tr><td>Environnement</td><td>SP.URB.GROW</td><td>Urban population growth (annual %)</td></tr><tr><td>Environnement</td><td>AG.SRF.TOTL.K2</td><td>Surface area (sq. km)</td></tr><tr><td>Environnement</td><td>EN.POP.DNST</td><td>Population density (people per sq. km of land area)</td></tr><tr><td>Environnement</td><td>EN.ATM.NOXE.KT.CE</td><td>Nitrous oxide emissions (thousand metric tons of CO2 equivalent)</td></tr><tr><td>Environnement</td><td>EN.ATM.METH.KT.CE</td><td>Methane emissions (kt of CO2 equivalent)</td></tr><tr><td>Environnement</td><td>AG.LND.AGRI.K2</td><td>Agricultural land (sq. km)</td></tr><tr><td>Environnement</td><td>EN.ATM.GHGO.KT.CE</td><td>Other greenhouse gas emissions, HFC, PFC and SF6 (thousand metric tons of CO2 equivalent)</td></tr><tr><td>Environnement</td><td>AG.LND.AGRI.ZS</td><td>Agricultural land (% of land area)</td></tr><tr><td>Environnement</td><td>AG.PRD.FOOD.XD</td><td>Food production index (2004-2006 = 100)</td></tr><tr><td>Environnement</td><td>AG.PRD.LVSK.XD</td><td>Livestock production index (2004-2006 = 100)</td></tr><tr><td>Environnement</td><td>AG.PRD.CROP.XD</td><td>Crop production index (2004-2006 = 100)</td></tr><tr><td>Environnement</td><td>EN.ATM.GHGT.KT.CE</td><td>Total greenhouse gas emissions (kt of CO2 equivalent)</td></tr><tr><td>Environnement</td><td>EN.ATM.CO2E.GF.ZS</td><td>CO2 emissions from gaseous fuel consumption (% of total)</td></tr><tr><td>Environnement</td><td>EN.ATM.CO2E.LF.ZS</td><td>CO2 emissions from liquid fuel consumption (% of total)</td></tr><tr><td>Environnement</td><td>EN.ATM.CO2E.SF.ZS</td><td>CO2 emissions from solid fuel consumption (% of total)</td></tr><tr><td>Environnement</td><td>NY.GDP.FRST.RT.ZS</td><td>Forest rents (% of GDP)</td></tr><tr><td>Environnement</td><td>NY.GDP.MINR.RT.ZS</td><td>Mineral rents (% of GDP)</td></tr><tr><td>Environnement</td><td>EN.ATM.CO2E.LF.KT</td><td>CO2 emissions from liquid fuel consumption (kt)</td></tr><tr><td>Environnement</td><td>EN.ATM.METH.AG.KT.CE</td><td>Agricultural methane emissions (thousand metric tons of CO2 equivalent)</td></tr><tr><td>Environnement</td><td>EN.ATM.METH.EG.KT.CE</td><td>Methane emissions in energy sector (thousand metric tons of CO2 equivalent)</td></tr><tr><td>Environnement</td><td>EN.ATM.NOXE.AG.KT.CE</td><td>Agricultural nitrous oxide emissions (thousand metric tons of CO2 equivalent)</td></tr><tr><td>Environnement</td><td>EN.ATM.NOXE.EG.KT.CE</td><td>Nitrous oxide emissions in energy sector (thousand metric tons of CO2 equivalent)</td></tr><tr><td>Environnement</td><td>NY.GDP.TOTL.RT.ZS</td><td>Total natural resources rents (% of GDP)</td></tr><tr><td>Environnement</td><td>EN.ATM.CO2E.KT</td><td>CO2 emissions (kt)</td></tr><tr><td>Environnement</td><td>EN.ATM.CO2E.PC</td><td>CO2 emissions (metric tons per capita)</td></tr><tr><td>Health</td><td>SP.POP.GROW</td><td>Population growth (annual %)</td></tr><tr><td>Health</td><td>SP.DYN.CBRT.IN</td><td>Birth rate, crude (per 1,000 people)</td></tr><tr><td>Health</td><td>SP.DYN.CDRT.IN</td><td>Death rate, crude (per 1,000 people)</td></tr><tr><td>Health</td><td>SP.DYN.TFRT.IN</td><td>Fertility rate, total (births per woman)</td></tr><tr><td>Health</td><td>SP.ADO.TFRT</td><td>Adolescent fertility rate (births per 1,000 women ages 15-19)</td></tr><tr><td>Health</td><td>SP.DYN.LE00.FE.IN</td><td>Life expectancy at birth, female (years)</td></tr><tr><td>Health</td><td>SP.DYN.LE00.IN</td><td>Life expectancy at birth, total (years)</td></tr><tr><td>Health</td><td>SP.POP.DPND</td><td>Age dependency ratio (% of working-age population)</td></tr><tr><td>Health</td><td>SP.DYN.AMRT.FE</td><td>Mortality rate, adult, female (per 1,000 female adults)</td></tr><tr><td>Health</td><td>SP.DYN.AMRT.MA</td><td>Mortality rate, adult, male (per 1,000 male adults)</td></tr><tr><td>Infrastructure</td><td>IT.CEL.SETS</td><td>Mobile cellular subscriptions</td></tr><tr><td>Infrastructure</td><td>IT.MLT.MAIN</td><td>Fixed telephone subscriptions</td></tr><tr><td>National_accounts</td><td>NY.ADJ.AEDU.GN.ZS</td><td>Adjusted savings: education expenditure (% of GNI)</td></tr><tr><td>National_accounts</td><td>NY.GDP.MKTP.CD</td><td>GDP (current USD)</td></tr><tr><td>National_accounts</td><td>NY.GDP.PCAP.CD</td><td>GDP per capita (current USD)</td></tr>
</table>  

In [2]:
df = pd.read_csv('df_sub_choix.csv', sep=',', low_memory=False)
df.drop(["Unnamed: 0"], axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1782448 entries, 0 to 1782447
Data columns (total 50 columns):
Cancer_code             object
Sex                     int64
Age_bucket              int64
Year                    int64
Country                 object
Mortality_rate          float64
SP.URB.TOTL.IN.ZS       float64
SP.URB.GROW             float64
SP.POP.GROW             float64
SE.PRM.DURS             float64
AG.SRF.TOTL.K2          float64
SE.PRE.DURS             float64
EN.POP.DNST             float64
SE.SEC.DURS             float64
SP.DYN.CBRT.IN          float64
SP.DYN.CDRT.IN          float64
NY.ADJ.AEDU.GN.ZS       float64
SP.DYN.TFRT.IN          float64
SP.ADO.TFRT             float64
SP.DYN.LE00.FE.IN       float64
SP.DYN.LE00.IN          float64
SP.POP.DPND             float64
SP.DYN.AMRT.FE          float64
SP.DYN.AMRT.MA          float64
EN.ATM.NOXE.KT.CE       float64
EN.ATM.METH.KT.CE       float64
AG.LND.AGRI.K2          float64
EN.ATM.GHGO.KT.CE       float64

In [3]:
df.head()

,Cancer_code,Sex,Age_bucket,Year,Country,Mortality_rate,SP.URB.TOTL.IN.ZS,SP.URB.GROW,SP.POP.GROW,SE.PRM.DURS,...,EN.ATM.CO2E.LF.KT,EN.ATM.METH.AG.KT.CE,EN.ATM.METH.EG.KT.CE,EN.ATM.NOXE.AG.KT.CE,EN.ATM.NOXE.EG.KT.CE,NY.GDP.TOTL.RT.ZS,EN.ATM.CO2E.KT,EN.ATM.CO2E.PC,NY.GDP.MKTP.CD,NY.GDP.PCAP.CD
0,C80,2,1,2000,Jamaica,1.433178,51.814,1.24118,0.776905,6.0,...,9856.896,633.005651,267.408083,361.597144,52.552558,1.762288,10307.937,3.964446,8.929376e+09,3434.249741
1,C56,2,1,2000,Jamaica,0.000000,51.814,1.24118,0.776905,6.0,...,9856.896,633.005651,267.408083,361.597144,52.552558,1.762288,10307.937,3.964446,8.929376e+09,3434.249741
2,C64,2,1,2000,Jamaica,0.716589,51.814,1.24118,0.776905,6.0,...,9856.896,633.005651,267.408083,361.597144,52.552558,1.762288,10307.937,3.964446,8.929376e+09,3434.249741
3,C44,2,1,2000,Jamaica,0.000000,51.814,1.24118,0.776905,6.0,...,9856.896,633.005651,267.408083,361.597144,52.552558,1.762288,10307.937,3.964446,8.929376e+09,3434.249741
4,C84,2,1,2000,Jamaica,0.000000,51.814,1.24118,0.776905,6.0,...,9856.896,633.005651,267.408083,361.597144,52.552558,1.762288,10307.937,3.964446,8.929376e+09,3434.249741


In [4]:
df.describe()

,Sex,Age_bucket,Year,Mortality_rate,SP.URB.TOTL.IN.ZS,SP.URB.GROW,SP.POP.GROW,SE.PRM.DURS,AG.SRF.TOTL.K2,SE.PRE.DURS,...,EN.ATM.CO2E.LF.KT,EN.ATM.METH.AG.KT.CE,EN.ATM.METH.EG.KT.CE,EN.ATM.NOXE.AG.KT.CE,EN.ATM.NOXE.EG.KT.CE,NY.GDP.TOTL.RT.ZS,EN.ATM.CO2E.KT,EN.ATM.CO2E.PC,NY.GDP.MKTP.CD,NY.GDP.PCAP.CD
count,1.782448e+06,1.782448e+06,1.782448e+06,1.782448e+06,1.782448e+06,1.782448e+06,1.782448e+06,1.782448e+06,1.782448e+06,1.782448e+06,...,1.782448e+06,1.782448e+06,1.782448e+06,1.782448e+06,1.782448e+06,1.782448e+06,1.782448e+06,1.782448e+06,1.782448e+06,1.782448e+06
mean,1.515686e+00,8.500000e+00,2.007045e+03,7.384622e+00,6.937213e+01,1.141402e+00,7.958524e-01,5.585801e+00,7.703769e+05,2.824726e+00,...,6.260160e+04,1.672510e+04,9.938026e+03,1.010188e+04,1.181828e+03,3.971666e+00,1.465975e+05,6.833518e+00,4.311381e+11,1.962986e+04
std,4.997540e-01,4.609774e+00,4.298007e+00,5.058222e+01,1.724218e+01,1.520220e+00,1.334384e+00,9.620440e-01,2.104753e+06,8.453246e-01,...,1.028553e+05,4.128721e+04,2.691539e+04,2.153634e+04,1.813369e+03,8.040714e+00,2.370915e+05,6.300614e+00,8.687206e+11,2.016993e+04
min,1.000000e+00,1.000000e+00,2.000000e+03,0.000000e+00,9.092000e+00,-2.697859e+00,-2.850973e+00,3.000000e+00,1.800000e+02,1.000000e+00,...,-4.781768e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.566900e+01,1.805539e-01,6.310127e+07,1.391091e+02
25%,1.000000e+00,4.750000e+00,2.003000e+03,0.000000e+00,5.780000e+01,2.848873e-01,1.427684e-01,5.000000e+00,4.289000e+04,2.000000e+00,...,6.750947e+03,1.154176e+03,8.483647e+02,9.688910e+02,9.943627e+01,1.946849e-01,1.216344e+04,2.513925e+00,2.316879e+10,4.274941e+03
50%,2.000000e+00,8.500000e+00,2.007000e+03,0.000000e+00,7.071500e+01,1.032221e+00,6.812672e-01,6.000000e+00,1.098900e+05,3.000000e+00,...,2.349814e+04,4.160227e+03,1.844764e+03,3.204051e+03,3.154942e+02,1.076101e+00,4.988954e+04,5.474267e+00,1.031348e+11,1.152759e+04
75%,2.000000e+00,1.225000e+01,2.011000e+03,1.260845e+00,8.263000e+01,1.842688e+00,1.348044e+00,6.000000e+00,4.067500e+05,3.000000e+00,...,5.475564e+04,1.580906e+04,1.209432e+04,9.628761e+03,1.437733e+03,3.560658e+00,1.699728e+05,8.953622e+00,3.533611e+11,3.044085e+04
max,2.000000e+00,1.600000e+01,2.015000e+03,8.091666e+03,1.000000e+02,1.789961e+01,1.762477e+01,8.000000e+00,1.709825e+07,4.000000e+00,...,6.613324e+05,3.159498e+05,4.067308e+05,1.574615e+05,8.035202e+03,8.092060e+01,1.768073e+06,6.745220e+01,6.203213e+12,1.175078e+05


In [5]:
#vérification
print("Base de données choix cancer : ")
print('- années incluses : ', np.sort(df.Year.unique()), '\n =>nb années :', len(df.Year.unique()))
print()
print('- pays inclus :', np.sort(df.Country.unique()), '\n =>nb pays :', len(df.Country.unique()) )
print()
print('- types de cancer inclus :', np.sort(df.Cancer_code.unique()), '\n =>nb types cancer :', len(df.Cancer_code.unique()) )

Base de données choix cancer : 
- années incluses :  [2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014
 2015] 
 =>nb années : 16

- pays inclus : ['Albania' 'Antigua and Barbuda' 'Argentina' 'Armenia' 'Aruba' 'Australia'
 'Austria' 'Azerbaijan' 'Bahrain' 'Barbados' 'Belarus' 'Belgium' 'Belize'
 'Bolivia' 'Bosnia and Herzegovina' 'Brazil' 'Brunei Darussalam' 'Bulgaria'
 'Canada' 'Chile' 'Colombia' 'Costa Rica' 'Croatia' 'Cuba' 'Cyprus'
 'Czech Republic' 'Denmark' 'Dominican Republic' 'Ecuador' 'El Salvador'
 'Estonia' 'Fiji' 'Finland' 'France' 'Georgia' 'Germany' 'Greece' 'Grenada'
 'Guatemala' 'Guyana' 'Haiti' 'Honduras' 'Hungary' 'Iceland' 'Iraq'
 'Ireland' 'Israel' 'Italy' 'Jamaica' 'Japan' 'Jordan' 'Kazakhstan'
 'Kiribati' 'Kuwait' 'Latvia' 'Lithuania' 'Luxembourg' 'Malaysia'
 'Maldives' 'Malta' 'Mauritius' 'Mexico' 'Montenegro' 'Morocco'
 'Netherlands' 'New Zealand' 'Nicaragua' 'Norway' 'Oman' 'Panama'
 'Paraguay' 'Peru' 'Philippines' 'Poland' 'Portugal' '

In [6]:
#variables explicatives quantitatives
indicators = ['SP.URB.TOTL.IN.ZS',	'SP.URB.GROW',	'SP.POP.GROW',	'SE.PRM.DURS',	'AG.SRF.TOTL.K2',	'SE.PRE.DURS',	'EN.POP.DNST',	'SE.SEC.DURS',	'SP.DYN.CBRT.IN',	'SP.DYN.CDRT.IN',	'NY.ADJ.AEDU.GN.ZS',	'SP.DYN.TFRT.IN',	'SP.ADO.TFRT',	'SP.DYN.LE00.FE.IN',	'SP.DYN.LE00.IN',	'SP.POP.DPND',	'SP.DYN.AMRT.FE',	'SP.DYN.AMRT.MA',	'EN.ATM.NOXE.KT.CE',	'EN.ATM.METH.KT.CE',	'AG.LND.AGRI.K2',	'EN.ATM.GHGO.KT.CE',	'AG.LND.AGRI.ZS',	'AG.PRD.FOOD.XD',	'IT.CEL.SETS',	'AG.PRD.LVSK.XD',	'AG.PRD.CROP.XD',	'IT.MLT.MAIN',	'EN.ATM.GHGT.KT.CE',	'EN.ATM.CO2E.GF.ZS',	'EN.ATM.CO2E.LF.ZS',	'EN.ATM.CO2E.SF.ZS',	'NY.GDP.FRST.RT.ZS',	'NY.GDP.MINR.RT.ZS',	'EN.ATM.CO2E.LF.KT',	'EN.ATM.METH.AG.KT.CE',	'EN.ATM.METH.EG.KT.CE',	'EN.ATM.NOXE.AG.KT.CE',	'EN.ATM.NOXE.EG.KT.CE',	'NY.GDP.TOTL.RT.ZS',	'EN.ATM.CO2E.KT',	'EN.ATM.CO2E.PC',	'NY.GDP.MKTP.CD',	'NY.GDP.PCAP.CD']          

#variables explicatives catégorielles : sex, tranche d'age, pays
df["Country_code"] = df["Country"].astype('category').cat.codes
df["Sex"] = df["Sex"].astype('category')
df["Age_bucket"] = df["Age_bucket"].astype('category')

features = [['Country_code', 'Sex', 'Age_bucket']+indicators]

In [7]:
nb_cancers = len(df['Cancer_code'].unique()) #nombre de types de cancers dans le dataset
cancer_types = list(df['Cancer_code'].unique()) #liste de types de cancers dans le dataset
#initialiser listes
lasso_cancer_codes = np.empty(nb_cancers, dtype=object)
lasso_r2 = np.empty(nb_cancers, dtype='float64')
lasso_nb_features = np.empty(nb_cancers, dtype='int')
i=0 

In [8]:
for c in cancer_types:

    X_scaled = scaler.fit_transform( df[df['Cancer_code'] == c][['Country_code', 'Sex', 'Age_bucket']+indicators])

    y = df[df['Cancer_code'] == c]['Mortality_rate']
 
    linlasso = Lasso(alpha=0.005, max_iter = -1).fit(X_scaled, y)  
    #!! Expliquer choix alpha, max_iter    
    
    lasso_cancer_codes[i] = c
    lasso_nb_features[i] = np.sum(linlasso.coef_ != 0)
    lasso_r2[i] = linlasso.score(X_scaled, y)
    i = i+1

df_lasso1 = pd.DataFrame({'Cancer_code':lasso_cancer_codes, 'R2':lasso_r2, 'Nb_features':lasso_nb_features}) 

Wall time: 1min 7s


In [10]:
#df_lasso1.to_csv(path_or_buf = 'lasso_choice_cancers.csv', sep=',', na_rep='')
print("Types de cancers mieux prédits par les variables explicatives : \n")
df_lasso1.sort_values(by=['R2'],inplace=False, ascending=False)

Types de cancers mieux prédits par les variables explicatives : 



,Cancer_code,Nb_features,R2
74,C14,31,0.393829
79,C15-C26,33,0.379968
83,C30-C39,29,0.373654
14,C71,31,0.367795
40,"C33,C34",45,0.335465
1,C56,36,0.329729
84,"C64,C65,C68",20,0.329394
76,"C40, C41, C50",26,0.323523
42,C25,36,0.316851
77,"C40, C41",15,0.310570


Il paraît que le type de cancer le mieux expliqué par les variables choisies et pendant le période observée serait C14 (pharynx), avec 33 features significatives et un score de 0.39.

Le cancer des poumons, C33,34 a un score très proche et 45 des variables sont significatives dans le modèle correspondant. On décide donc de préférer ce dernier par la suite.

In [11]:
#détails du modèle optimal
c_series = df_lasso1[df_lasso1['R2'] == max(df_lasso1['R2'])]['Cancer_code'] #type de cancer avec le meilleur score R2
c_best = list(c_series)[0]
X_scaled = scaler.fit_transform( df[df['Cancer_code'] == c_best][['Country_code', 'Sex', 'Age_bucket']+indicators] )
y = df[df['Cancer_code'] == c_best]['Mortality_rate']
linlasso = Lasso(alpha=0.005, max_iter = -1).fit(X_scaled, y)  
print("Résultats du modèle lasso sur le cancer le mieux expliqué : ")
print()
print('Type de cancer : ', c_best)
print('R2 : ', linlasso.score(X_scaled, y) )
y_pred = linlasso.predict(X_scaled)
rmse = sqrt(mean_squared_error(y, y_pred)); print('RMSE : ', rmse)
print('Nombre de features prises en compte par le modèle : ', np.sum(linlasso.coef_ != 0))
print('Features ordonnées par ordre décroissante de leur importance dans le modèle :')
for e in sorted (list(zip(list(df[['Country_code', 'Sex', 'Age_bucket']+indicators]), linlasso.coef_)),
                key = lambda e: -abs(e[1])):
    if e[1] != 0:
        print('\t{}, {:.3f}'.format(e[0], e[1]))   

Résultats du modèle lasso sur le cancer le mieux expliqué : 

Type de cancer :  C14
R2 :  0.393829314774
RMSE :  16.746780924639626
Nombre de features prises en compte par le modèle :  31
Features ordonnées par ordre décroissante de leur importance dans le modèle :
	Age_bucket, 37.295
	SE.PRM.DURS, 12.953
	SP.ADO.TFRT, 12.188
	AG.LND.AGRI.ZS, -11.935
	Sex, -9.509
	SE.SEC.DURS, 8.941
	SP.POP.DPND, -8.771
	SP.DYN.AMRT.FE, 8.299
	AG.PRD.CROP.XD, -7.712
	EN.ATM.CO2E.LF.ZS, 6.868
	SP.DYN.LE00.FE.IN, 6.698
	SP.DYN.TFRT.IN, -6.650
	EN.ATM.METH.EG.KT.CE, 6.374
	NY.ADJ.AEDU.GN.ZS, 5.511
	AG.LND.AGRI.K2, 5.489
	EN.ATM.NOXE.AG.KT.CE, -5.426
	AG.PRD.FOOD.XD, 5.309
	NY.GDP.TOTL.RT.ZS, -4.191
	EN.ATM.NOXE.EG.KT.CE, -3.766
	EN.ATM.CO2E.LF.KT, 3.493
	EN.ATM.CO2E.PC, 3.385
	EN.ATM.GHGO.KT.CE, 2.255
	EN.ATM.CO2E.GF.ZS, 2.201
	NY.GDP.FRST.RT.ZS, -1.698
	EN.POP.DNST, 1.601
	NY.GDP.MINR.RT.ZS, -1.278
	SP.URB.TOTL.IN.ZS, -1.156
	SP.DYN.AMRT.MA, 0.625
	SP.POP.GROW, 0.405
	IT.CEL.SETS, -0.219
	SP.URB.GROW, -0

In [12]:
#détails du modèle pour le cancer des poumons
c = 'C33,C34' #code cancer des poumons
X_scaled = scaler.fit_transform( df[df['Cancer_code'] == c][['Country_code', 'Sex', 'Age_bucket']+indicators] )
y = df[df['Cancer_code'] == c]['Mortality_rate']
linlasso = Lasso(alpha=0.005, max_iter = -1).fit(X_scaled, y)  
print("Résultats du modèle lasso sur le cancer des poumons : ")
print()
print('Type de cancer : ', c)
print('R2 : ', linlasso.score(X_scaled, y) )
y_pred = linlasso.predict(X_scaled)
rmse = sqrt(mean_squared_error(y, y_pred)); print('RMSE : ', rmse)
print('Nombre de features prises en compte par le modèle : ', np.sum(linlasso.coef_ != 0))
print('Features ordonnées par ordre décroissante de leur importance dans le modèle :')
for e in sorted (list(zip(list(df[['Country_code', 'Sex', 'Age_bucket']+indicators]), linlasso.coef_)),
                key = lambda e: -abs(e[1])):
    if e[1] != 0:
        print('\t{}, {:.3f}'.format(e[0], e[1]))   

Résultats du modèle lasso sur le cancer des poumons : 

Type de cancer :  C33,C34
R2 :  0.33546481578
RMSE :  107.99438667700235
Nombre de features prises en compte par le modèle :  45
Features ordonnées par ordre décroissante de leur importance dans le modèle :
	Age_bucket, 221.040
	AG.SRF.TOTL.K2, 65.065
	EN.ATM.METH.EG.KT.CE, -61.157
	SP.URB.GROW, -56.592
	SP.DYN.LE00.IN, 52.084
	EN.ATM.METH.KT.CE, 51.877
	Sex, -51.483
	SP.DYN.CDRT.IN, 40.758
	AG.LND.AGRI.K2, -40.178
	SP.DYN.TFRT.IN, -35.366
	EN.ATM.CO2E.SF.ZS, 34.422
	SP.URB.TOTL.IN.ZS, 30.745
	SP.POP.GROW, 29.483
	EN.ATM.CO2E.LF.KT, -29.474
	EN.ATM.CO2E.GF.ZS, 27.813
	EN.ATM.NOXE.AG.KT.CE, -26.836
	SE.PRM.DURS, 25.251
	SE.SEC.DURS, 21.344
	SP.DYN.CBRT.IN, 21.234
	SP.ADO.TFRT, -21.019
	IT.CEL.SETS, -18.597
	NY.GDP.MKTP.CD, 17.465
	AG.LND.AGRI.ZS, 17.026
	EN.ATM.CO2E.LF.ZS, 14.335
	NY.GDP.FRST.RT.ZS, 13.884
	EN.ATM.GHGO.KT.CE, -13.618
	SP.DYN.AMRT.FE, 12.547
	EN.ATM.NOXE.EG.KT.CE, 12.273
	NY.GDP.TOTL.RT.ZS, -12.249
	EN.ATM.CO2E.PC, 

#### Choix de décalage optimal entre l'année d'observation des indicateurs socio-économiques et l'année d'observation du taux de mortalité (delta)
Ayant choisi notre cancer à expliquer, on continue à chercher la différence d'années la plus pertinente. 

A chaque itération, on construit une base de données avec les variables explicatives pour l'année 2000 et une variable expliquée décalée de 'delta' nombre d'années(Mortality_shifted). 

In [15]:
#séléctionner le cancer des poumons
df2 = df[(df['Cancer_code'] == 'C33,C34') ].copy()

#vérification
print("Base de données choix delta : ")
print('- années incluses : ', np.sort(df2.Year.unique()), '\n =>nb années :', len(df2.Year.unique()))
print()
print('- pays inclus :', np.sort(df2.Country.unique()), '\n =>nb pays :', len(df2.Country.unique()) )
print()
print('- types de cancer inclus :', np.sort(df2.Cancer_code.unique()), '\n =>nb types cancer :', len(df2.Cancer_code.unique()) )

Base de données choix delta : 
- années incluses :  [2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014
 2015] 
 =>nb années : 16

- pays inclus : ['Albania' 'Antigua and Barbuda' 'Argentina' 'Armenia' 'Aruba' 'Australia'
 'Austria' 'Azerbaijan' 'Bahrain' 'Barbados' 'Belarus' 'Belgium' 'Belize'
 'Bolivia' 'Bosnia and Herzegovina' 'Brazil' 'Brunei Darussalam' 'Bulgaria'
 'Canada' 'Chile' 'Colombia' 'Costa Rica' 'Croatia' 'Cuba' 'Cyprus'
 'Czech Republic' 'Denmark' 'Dominican Republic' 'Ecuador' 'El Salvador'
 'Estonia' 'Fiji' 'Finland' 'France' 'Georgia' 'Germany' 'Greece' 'Grenada'
 'Guatemala' 'Guyana' 'Haiti' 'Honduras' 'Hungary' 'Iceland' 'Iraq'
 'Ireland' 'Israel' 'Italy' 'Jamaica' 'Japan' 'Jordan' 'Kazakhstan'
 'Kiribati' 'Kuwait' 'Latvia' 'Lithuania' 'Luxembourg' 'Malaysia'
 'Maldives' 'Malta' 'Mauritius' 'Mexico' 'Montenegro' 'Morocco'
 'Netherlands' 'New Zealand' 'Nicaragua' 'Norway' 'Oman' 'Panama'
 'Paraguay' 'Peru' 'Philippines' 'Poland' 'Portugal' 'P

In [16]:
#initialisation des listes
max_delta = max(df2['Year']) - min(df2['Year'])
year = 2000
lasso_base_year = np.empty(max_delta, dtype='int') #année des variables explicatives
lasso_delta = np.empty(max_delta, dtype='int') #année du taux de mortalité : year + delta
lasso_nb_obs = np.empty(max_delta, dtype='int') #nombre d'observations prises en compte par le modèle
lasso_r2 = np.empty(max_delta, dtype='float64')
lasso_nb_features = np.empty(max_delta, dtype='int')
i=0

In [17]:
for delta in range(0,max_delta):
    df2['Mortality_shifted'] = df2.groupby(['Country', 'Sex', 'Age_bucket'])['Mortality_rate'].shift(-delta)
    y = df2[df2['Year'] == 2000].dropna()['Mortality_shifted']
    X_scaled = scaler.fit_transform( df2[df2['Year'] == year].dropna()[indicators + ['Country_code', 'Sex', 'Age_bucket']] )
    linlasso = Lasso(alpha=0.005, max_iter = 10000).fit(X_scaled, y)  

    lasso_base_year[i] = year
    lasso_delta[i] = delta    
    lasso_nb_obs[i] = len(y)               
    lasso_nb_features[i] = np.sum(linlasso.coef_ != 0)
    lasso_r2[i] = linlasso.score(X_scaled, y)
    i = i+1

df_lasso = pd.DataFrame({'Base_year':lasso_base_year,
                         'Delta':lasso_delta, 
                         'Nb_obs': lasso_nb_obs,
                         'Nb_features':lasso_nb_features,
                         'R2': lasso_r2 })   

In [18]:
df_lasso.sort_values(by=['R2'], ascending=False)

,Base_year,Delta,Nb_features,Nb_obs,R2
14,2000,14,17,256,0.384543
13,2000,13,21,608,0.376205
12,2000,12,33,1008,0.363423
11,2000,11,43,1504,0.359705
0,2000,0,41,2704,0.357286
5,2000,5,38,2048,0.351923
10,2000,10,41,1792,0.351338
9,2000,9,39,1888,0.346783
2,2000,2,42,2432,0.346114
6,2000,6,43,2048,0.344600


In [19]:
#Détails pour le meilleur modèle en termes de décalage
delta_series = df_lasso[df_lasso['R2'] == max(df_lasso['R2'])]['Delta'] #meilleur delta selon R2
delta = list(delta_series)[0]

df2['Mortality_shifted'] = df2.groupby(['Sex', 'Age_bucket', 'Country'])['Mortality_rate'].shift(-delta)
y = df2[df2['Year'] == 2000].dropna()['Mortality_shifted']
X_scaled = scaler.fit_transform( df2[df2['Year'] == year].dropna()[indicators + ['Sex', 'Age_bucket']] )
linlasso = Lasso(alpha=0.005, max_iter = 10000).fit(X_scaled, y)   
print("Résultats du modèle lasso sur le cancer des poumons avec décalage dans le temps : ")
print()
print('Delta : ', delta)
print('R2 : ', linlasso.score(X_scaled, y) )
y_pred = linlasso.predict(X_scaled)
rmse = sqrt(mean_squared_error(y, y_pred)); print('RMSE : ', rmse)
print('Nombre de features prises en compte par le modèle : ', np.sum(linlasso.coef_ != 0))
print('Features ordonnées par coefficients :')
for e in sorted (list(zip(list(df2[indicators+['Country_code', 'Sex', 'Age_bucket']]), linlasso.coef_)),
                key = lambda e: -abs(e[1])):
    if e[1] != 0:
        print('\t{}, {:.3f}'.format(e[0], e[1]))   
        


Résultats du modèle lasso sur le cancer des poumons avec décalage dans le temps : 

Delta :  14
R2 :  0.384543090178
RMSE :  112.97300238504384
Nombre de features prises en compte par le modèle :  17
Features ordonnées par coefficients :
	Sex, 254.529
	Country_code, -58.263
	EN.POP.DNST, 39.535
	SP.URB.GROW, -22.127
	IT.CEL.SETS, 21.094
	SE.PRE.DURS, 17.395
	SP.DYN.CBRT.IN, -15.239
	SP.ADO.TFRT, -7.248
	SP.POP.GROW, -2.805
	NY.GDP.FRST.RT.ZS, -2.403
	AG.LND.AGRI.ZS, 2.158
	SE.PRM.DURS, -1.943
	EN.ATM.CO2E.KT, 1.789
	EN.ATM.CO2E.GF.ZS, 1.049
	SE.SEC.DURS, 0.942
	IT.MLT.MAIN, 0.794
	AG.PRD.LVSK.XD, 0.001


Le meilleur résultat est obtenu pour un delta = 14, c'est à dire les indicateurs socio-économiques de l'année 2000 expliquent le mieux le taux de mortalité de l'année 2014 pour le type de cancer sélectionné.

Cependant, le nombre d'observations pour ce décalage est très réduit, puisque de moins en moins d'observations sont disponibles pour les années les plus récentes. En plus, seules 17 features sont retenues, ce qui pourrait indiquer un fort sur-apprentissage.

Les résultats des modèles avec prise en compte de décalage dans le temps sont assez proches, donc nous décidons de <b>ne pas appliquer un décalage</b> pour les autres modèles qu'on essayera d'optimiser.